# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
pip install cartopy geoviews pyproj

     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ---------------------------------------- 56.0/56.0 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of panel to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.2/11.0 MB 4.6 MB/s eta 0:00:03
    --------------------------------------- 0.3/11.0 MB 3.3 MB/s eta 0:00:04
   - -------------------------------------- 0.5/11.0 MB 3.5 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/11.0 MB 3.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.0 MB 3.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.0 MB 3.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/11.0 MB 3.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/11.0 MB 3.7 MB/s eta 0:00:03
   ----- -----------

In [11]:
# Dependencies and Setup.
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,callao,-12.0667,-77.1500,22.16,83,0,6.69,PE,1737591704
1,1,carolina beach,34.0352,-77.8936,-2.74,66,99,5.69,US,1737591706
2,2,suong,11.9115,105.6533,20.65,55,100,0.61,KH,1737591707
3,3,lebu,-37.6167,-73.6500,15.25,83,33,0.76,CL,1737591450
4,4,adamstown,-25.0660,-130.1015,25.02,79,8,3.85,PN,1737591709


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    x="Lng",                     # Longitude for x-axis
    y="Lat",                     # Latitude for y-axis
    size="Humidity",             # Point size based on Humidity
    color="Humidity",            # Point color based on Humidity
    geo=True,                    # Enable geographic plotting
    tiles="CartoLight",          # Use a light map background
    frame_width=800,             # Width of the plot
    frame_height=600,            # Height of the plot
    title="City Humidity Map"    # Title of the map
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[
    (city_data_df["Max Temp"] >= 2) &
    (city_data_df["Max Temp"] <= 90) &
    (city_data_df["Humidity"] < 90) &
    (city_data_df["Cloudiness"] < 90)
]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities.head()
ideal_cities.shape

(212, 10)

### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()



,City,Country,Lat,Lng,Humidity,Hotel Name
0,callao,PE,-12.0667,-77.1500,83,
3,lebu,CL,-37.6167,-73.6500,83,
4,adamstown,PN,-25.0660,-130.1015,79,
5,utrik,MH,11.2278,169.8474,78,
6,uturoa,PF,-16.7333,-151.4333,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
# Set search radius (10,000 meters)
radius = 10000

# Base URL for Geoapify API
base_url = "https://api.geoapify.com/v2/places"

# Define parameters for the API search
params = {
    "categories": "accommodation.hotel",  # Looking for hotels
    "apiKey": geoapify_key,  # Your Geoapify API key
    "limit": 1  # Limit to the first result
}

In [17]:
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    print (lat) 
    print (lng) 
   # Add latitude and longitude to the filter parameter
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Make an API request
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    print(name_address)

    # Grab the first hotel from the results and store the name in the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    

Starting hotel search
-12.0667
-77.15
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Casa Ronald', 'country': 'Peru', 'country_code': 'pe', 'state': 'Constitutional Province of Callao', 'county': 'Callao', 'city': 'Lima Metropolitan Area', 'postcode': '07021', 'district': 'Castilla-Pasamayo', 'street': 'Calle La Constitución', 'lon': -77.14733012859645, 'lat': -12.0601567, 'formatted': 'Casa Ronald, Calle La Constitución, Lima Metropolitan Area 07021, Peru', 'address_line1': 'Casa Ronald', 'address_line2': 'Calle La Constitución, Lima Metropolitan Area 07021, Peru', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details.facilities'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Casa Ronald', 'osm_id': 469736257, 'tourism': 'hotel', 'buildi

In [8]:
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,callao,PE,-12.0667,-77.1500,83,Casa Ronald
3,lebu,CL,-37.6167,-73.6500,83,No hotel found
4,adamstown,PN,-25.0660,-130.1015,79,No hotel found
5,utrik,MH,11.2278,169.8474,78,No hotel found
6,uturoa,PF,-16.7333,-151.4333,77,Hawaiki Nui hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
import hvplot.pandas

# Add hover information to the DataFrame
hotel_df["Hover Info"] = (
    "City: " + hotel_df["City"] +
    "Country: " + hotel_df["Country"] +
    "Hotel: " + hotel_df["Hotel Name"] +
    "Humidity: " + hotel_df["Humidity"].astype(str) + "%"
)

# Create a map with the hover information
city_map = hotel_df.hvplot.points(
    "Lng", "Lat",  # Longitude and Latitude for the map
    geo=True,  # Enable geographic plotting
    tiles="CartoLight",  # Map background
    color="Humidity",  # Color based on Humidity
    size=50,  # Size of the points
    hover_cols=["Hover Info"],  # Add the hover information
    title="City Map with Hotel Information"
)

# Display the map
city_map




:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hover Info)